In [1]:
import aylien_news_api
from aylien_news_api.rest import ApiException
from pprint import pprint
import copy
import dotenv
import json
import os
import time
import requests

In [2]:
# Configure API key authorization: app_id
app_id = dotenv.get_key('../.env', 'AYLIEN_APP_ID')
app_key = dotenv.get_key('../.env', 'AYLIEN_APP_KEY')
endpoint = dotenv.get_key('../.env', 'AYLIEN_ENDPOINT')

configuration = aylien_news_api.Configuration()
configuration.api_key['X-AYLIEN-NewsAPI-Application-ID'] = app_id
configuration.api_key['X-AYLIEN-NewsAPI-Application-Key'] = app_key
configuration.host = endpoint

In [3]:
client = aylien_news_api.ApiClient(configuration)
api_instance = aylien_news_api.DefaultApi(client)

In [25]:
# Fetch stories using AYLIEN News API
def fetch_stories_to_file(opts, append_to_file, limit=None):
    response = None
    fetched_stories = []
    stories = None
    RETRY_COUNT = 5
    MAX_PER_PAGE = 100
    retry = 0

    def _end_of_file():
        if append_to_file:
            with open(append_to_file, 'rb+') as f:
                # Remove trailing comma if present
                f.seek(-2, os.SEEK_END)
                if str(f.read(1), 'utf-8') == ',':
                    print("Removing trailing comma.")
                    f.seek(-1, os.SEEK_CUR)
                    f.truncate()
                    f.write(bytes('\n]\n', 'utf-8'))

    if append_to_file and os.path.exists(append_to_file):
        with open(append_to_file, 'r+') as f:
            if (start := f.read(1)) == '[':
                print("append_to non-empty.")
            elif start:
                print("append_to non-empty, but doesn't start with '['. Stopping.")
                return fetched_stories
            else:
                print("append_to empty. Will overwrite")
                f.write('[\n')

        with open(append_to_file, 'rb+') as f:
            f.seek(-2, os.SEEK_END)
            final = str(f.read(1), 'utf-8')
            f.seek(-2, os.SEEK_CUR)
            if final == ']':
                print("append_to ends with ']'. Removing.")
                f.truncate()
                f.write(bytes(',\n', 'utf-8'))
            elif final == ',' or final == '[':
                print("append_to ends with " + final + ". Will append to end.")
            else:
                print("append_to doesn't end with ',' or ']'. Check that it's valid JSON and that the file ends with newline. Stopping.")
                return fetched_stories

    while stories is None or len(stories) == (opts.get('per_page') or MAX_PER_PAGE):
        if limit and len(fetched_stories) >= limit:
            print('Reached limit of %d stories' % limit)
            print('Next page cursor: %s' % response.next_page_cursor or 'None')
            _end_of_file()
            return fetched_stories
        try:
            response = api_instance.list_stories(**opts)
        except ApiException as e:
            if e.status == 429:
                if int(e.headers['x-ratelimit-volume-remaining']) == 0:
                    reset = e.headers('x-ratelimit-volume-reset')
                    print('Monthly rate limit exceeded. Wait until reset at %s' % reset)
                    print('Next page cursor: %s' % response.next_page_cursor or 'None')
                    _end_of_file()
                    return fetched_stories
                if int(e.headers['x-ratelimit-remaining']) == 0:
                    print('1-Minute rate limit exceeded. Waiting 60 seconds...')
                    time.sleep(60)
                    continue
            else:
                print("Exception when calling DefaultApi->list_stories (see below)")
                print("Retry %d/%d" % (retry, RETRY_COUNT))
                print("Status: %d" % e.status)
                print("Reason: %s" % e.reason)
                if retry < RETRY_COUNT:
                    print("Retrying in 10 seconds...")
                    retry += 1
                    time.sleep(10)
                    continue
                else:
                    print("Giving up. Check end of file '%s' for partial results." % append_to_file)
                    print('Next page cursor: %s' % response.next_page_cursor or 'None')
                    _end_of_file()
                    return fetched_stories

        if append_to_file:
            with open(append_to_file, 'a') as f:
                for story in response.stories:
                    sentiment_dict = {'body_polarity': story.sentiment.body.polarity if story.sentiment.body else None,
                                      'body_score': story.sentiment.body.score if story.sentiment.body else None,
                                      'title_polarity': story.sentiment.title.polarity if story.sentiment.title else None,
                                      'title_score': story.sentiment.title.score if story.sentiment.title else None}

                    source_dict = {
                        'description': story.source.description if story.source else None,
                        'discriminator': story.source.discriminator if story.source else None,
                        'domain': story.source.domain if story.source else None,
                        'home_page_url': story.source.home_page_url if story.source else None,
                        'id': story.source.id if story.source else None,
                        'links_in_count': story.source.links_in_count if story.source else None
                    }

                    story_dict = {'document_id': story.id, 'body': story.body, 'date': story.published_at.isoformat(),
                                  'sentiment': sentiment_dict, 'source': source_dict}
                    fetched_stories.append(story_dict)
                    json.dump(story_dict, f, indent=4)
                    f.write(',\n')

        stories = response.stories
        opts['cursor'] = response.next_page_cursor

        print(f'Fetched {len(stories)} stories. Total: {len(fetched_stories)} so far.')

    _end_of_file()
    return fetched_stories

# Alternate: Fetch stories using pure HTTP requests
def fetch_news(api_id, api_key, opts):
    base_url = 'https://api.aylien.com/news/stories'
    headers = {
        'X-AYLIEN-NewsAPI-Application-ID': api_id,
        'X-AYLIEN-NewsAPI-Application-Key': api_key
    }
    params = {
        'published_at.start': opts['published_at_start'],
        'published_at.end': opts['published_at_end'],
        # 'source_locations.country': ','.join(opts['source_locations_country']),
        # 'language': ','.join(opts['language']),
        'sort_by': opts['sort_by'],
        # 'sort_direction': opts['sort_direction'],
        'per_page': opts['per_page'],
    }
    if 'aql' in opts:
        params['aql'] = opts['aql']
    elif 'text' in opts:
        params['text'] = opts['text']

    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error when calling AYLIEN News API: {response.status_code}, {response.text}")
        return None

In [5]:
# generate AYLIEN Query Language (AQL, based on Lucene) query from keywords
keywords_file = '../models/story_keywords.txt'
with open(keywords_file, 'r') as f:
    keywords = f.read().splitlines()

climate_keywords_aql = 'text:(' + ' OR '.join([f'"{keyword}"' for keyword in keywords]) + ')'

# generate AQL query from bill info
bill_data_file = '../data/bill_data.json'
with open(bill_data_file, 'r') as f:
    bill_data = json.load(f)

bill_aql_tuples = [(f'"{bill["short_title"]}"', f'"{bill["bill_number"]}"') for bill in bill_data.values()]
half = len(bill_aql_tuples) // 2

bill_aql_1 = 'text:(' + ' OR '.join([st + ' OR ' + bn for st, bn in bill_aql_tuples[:half]]) + ')'
bill_aql_2 = 'text:(' + ' OR '.join([st + ' OR ' + bn for st, bn in bill_aql_tuples[half:]]) + ')'

In [6]:
# set options
opts_1 = {
    'aql': bill_aql_1,
    'published_at_start': 'NOW-27MONTH',
    'published_at_end': 'NOW-1DAY',
    'source_locations_country': ['US'],
    'language': ['en'],
    'sort_by': 'relevance',
    # 'sort_direction': 'asc',
    'per_page': 100,
}

opts_2 = copy.deepcopy(opts_1)
opts_2['aql'] = bill_aql_2

In [24]:
fetched = fetch_stories_to_file(opts_1, append_to_file='../data/fetched_stories.json', limit=3000)
# TODO: fetch for opts_2

append_to non-empty.
append_to ends with ']'. Removing.
Fetched 100 stories. Total: 100 so far.
Fetched 100 stories. Total: 200 so far.
Reached limit of 200 stories
Next page cursor: MjEwLjIwOTA5LDM4MDE3NDQ1NTc=
Removing trailing comma.
